## poging to emg analyse in time domain

**to do:**
- container voor regressie --> andere te gerbuiken
- verschil tussen de median frequency en mean frequency --> tijd - frequency domein? 
- reps vinden --> emg_filter.py?


In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib qt
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from analysing import AnalysisContainer, AnalysingResults
import os

trainings = {
    0: 0,
    1: 'LE_HYP_5x12',
    2: 'LE_HYP_8x12',
    3: 'LE_MAX_5x4',
    4: 'LP_HYP_5x12'   
}

folder_path = 'raw_emg_csv'


In [2]:
df = pd.read_csv('./raw_emg_csv/all_emg.csv')
df['training_id'] = df['training_id'].astype(int)
df['training_id'] = df['training_id'].map(trainings)
df.head(2)


C:\Users\Simon\AppData\Local\Temp\ipykernel_34440\187075156.py:1: DtypeWarning: Columns (19) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('./raw_emg_csv/all_emg.csv')


,Unnamed: 0.1,Unnamed: 0,time,vl,rf,vm,Filtered_rf,Filtered_vl,Filtered_vm,rf_abs,vl_abs,vm_abs,Rolling_RMS_rf,Rolling_RMS_vl,Rolling_RMS_vm,training_id,rep,set,pp,rep_busy
0,0,0,1970-01-01 00:00:00.000000000,5.36975,-0.158505,-0.438854,0.241542,-0.525227,0.171729,0.241542,0.525227,0.171729,1.739555,2.964063,1.479010,LE_HYP_5x12,0.0,0,1,NaN
1,1,1,1970-01-01 00:00:00.000700000,1.70965,-0.792526,0.172208,-0.020939,-3.883819,1.034977,0.020939,3.883819,1.034977,1.731612,2.987389,1.474593,LE_HYP_5x12,0.0,0,1,NaN


In [3]:
df[[p for p in df.columns if p.startswith('Rolling') or p in ['set', 'rep', 'pp', 'training_id']]].groupby(['pp', 'rep', 'set', 'training_id']).max().reset_index().to_csv('time_domain.csv')

In [54]:
#def plot_training(training_id:str, t_set:int, pp:int, rep:int, var='Rolling_RMS_vl', **kwargs):
#    if var not in df.columns:
#        raise KeyError(f'{var} not in {list(df.columns)}')
#    if t_set not in set(df['set']):
#        raise KeyError(f'{t_set} not in {set(df["set"])}')
#    if pp not in set(df['pp']):
#        raise KeyError(f'{pp} not in {pp(df["pp"])}')
#    if training_id not in set(df['training_id']):
#        raise KeyError(f'{training_id} not in {set(df["training_id"])}')
#    if rep not in set(df['rep']):
#        raise KeyError(f'{rep} not in {set(df["rep"])}')  
#
#    df_filtered = df[(df['training_id'] == training_id) & (df['set'] == t_set) & (df['pp'] == pp) & (df['rep'] == rep)]
#    
#    fig = go.Figure()
#    fig.add_trace(go.Scatter(x=df_filtered['time'], y=df_filtered[var], mode='lines', name=var))
#    fig.update_layout(title=f'Training Data for Set {t_set}, Participant {pp}', xaxis_title='Time', yaxis_title=var)
#    fig.show()
#    
#plot_training('1', 0, 1, 0, var='Rolling_RMS_vm')

In [4]:
df_calibration = df[(df['set'] == 0) & (df['Rolling_RMS_vl']>20)]
df_training = df[df['set']!=0]
#df[[column for column in df.columns if column.startswith('Rolling') or column in ['training_id', 'set', 'training_id', 'pp']]].groupby(['pp', 'training_id', 'set']).mean()
pivot = df_calibration[[column for column in df.columns if column.startswith('Rolling') or column in ['training_id', 'set', 'training_id', 'pp']]].groupby(['pp', 'training_id']).agg(['mean', 'std']).reset_index()


c_map = {
    'LE_HYP_5x12': 'green',
    'LE_HYP_8x12': 'red',
    'LE_MAX_5x4': 'blue',
    'LP_HYP_5x12': 'purple'   
}

#df_calibration = df_calibration[~((df_calibration['training_id'] != 'LE_HYP_5x12') & (df_calibration['pp'] == 4))]


sns.boxplot(data=df_calibration, x='pp', y='Rolling_RMS_rf', hue='training_id', dodge=True, linewidth=1, palette=c_map)
# Add legend
plt.legend(title='Training')
df_calibration.columns

Index(['Unnamed: 0.1', 'Unnamed: 0', 'time', 'vl', 'rf', 'vm', 'Filtered_rf',
       'Filtered_vl', 'Filtered_vm', 'rf_abs', 'vl_abs', 'vm_abs',
       'Rolling_RMS_rf', 'Rolling_RMS_vl', 'Rolling_RMS_vm', 'training_id',
       'rep', 'set', 'pp', 'rep_busy'],
      dtype='object')

**RMS correction for MVC** 

In [5]:
diff_types_training = set(df_training['training_id'])
#for training_id in diff_types_training:
#    for pp in df_training['pp'].unique():
#        calibration = pivot[(pivot['training_id'] == training_id) & (pivot['pp']==pp)]
#        for column in [column for column in df_training.columns if column.startswith('Rolling')]:
#            ratio = calibration[column].iloc[0]
#            df_training.loc[(df_training['training_id'] == training_id) & (df_training['pp'] == pp), column] /= ratio
        

In [6]:
container = AnalysisContainer()
predictors = [column for column in df_training.columns if column.startswith('Rolling')]

diff_types_training = ['LE_HYP_8x12']

for predictor in predictors: 
    fig, axs = plt.subplots(4, 8, figsize=(15, 15), sharey=True)
    plt.subplots_adjust(wspace=0.5, hspace=0.5)
    for i, training in enumerate(diff_types_training):
        results_avg = AnalysingResults(f'avg_{predictor}_{training}')
        results_max = AnalysingResults(f'max_{predictor}_{training}')
        
        training_filter = (df_training['training_id'] == training)
        t_sets = set(df_training[training_filter]['set'])
        filtered_df = df_training[training_filter]
        filtered_df = filtered_df[filtered_df['rep']>0]
        
        for j, t_set in enumerate(t_sets):
            df_to_process = filtered_df[filtered_df['set'] == t_set]
            grouped = df_to_process.groupby(['rep', 'pp'])
            avg = grouped.mean(numeric_only=True).reset_index()
            mx = grouped.max().reset_index()
            
            ax = axs[i, j]
            
            for pp in set(mx['pp']):
                #aanpassen
                pp=3
                df_pp_avg = avg[avg['pp']==pp]
                df_pp_max = mx[mx['pp']==pp]
                results_max.update(pp, training, t_set, f'max', df_pp_max, predictor, plot=False)
                results_avg.update(pp, training, t_set, f'avg', df_pp_avg, predictor, plot=True, ax=ax,title=f'set {t_set}', y_label='average magnitude')
                
                break
        
    
        container.update([results_max, results_avg])
    plt.suptitle(f'{predictor}', fontsize=20)
   
plt.show()
                

output1 = container.df_with_diffrent_columns
output2 = container.df_with_type_column

c:\Users\Simon\Documents\Ugent\Master\master proef\jupyter\emg\analysing.py:30: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  B1, B0 = model.params[1], model.params[0]
c:\Users\Simon\Documents\Ugent\Master\master proef\jupyter\emg\analysing.py:63: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  self.results['B0_std'].append(results[2][0])
c:\Users\Simon\Documents\Ugent\Master\master proef\jupyter\emg\analysing.py:65: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by pos

In [10]:
avg_tabel = output1.groupby(['training','set', 'type_analysis']).agg(['mean', 'std']).reset_index()
avg_tabel


training set type_analysis        pp                 B0_rf              \
                                       mean       std        mean         std   
0   LE_HYP_5x12   1           avg  2.500000  1.167748  171.670521   85.436674   
1   LE_HYP_5x12   1           max  2.500000  1.167748  274.822435  111.855803   
2   LE_HYP_5x12   2           avg  3.000000  1.463850  185.340255  109.924056   
3   LE_HYP_5x12   2           max  3.000000  1.463850  330.335131  171.000824   
4   LE_HYP_5x12   3           avg  3.000000  1.463850  187.601661   94.343383   
5   LE_HYP_5x12   3           max  3.000000  1.463850  333.318837  156.884037   
6   LE_HYP_5x12   4           avg  2.500000  1.167748  218.121772  101.332263   
7   LE_HYP_5x12   4           max  2.500000  1.167748  322.515045  137.961405   
8   LE_HYP_5x12   5           avg  3.000000  1.463850  195.986827  102.640261   
9   LE_HYP_5x12   5           max  3.000000  1.463850  329.710435  153.485241   
10  LE_HYP_8x12   1           avg  3.000000  1.463850  143.724841   34.580326   
11  LE_HYP_8x12   1           max  3.000000  1.463850  237.666724   37.102501   
12  LE_HYP_8x12   2           avg  3.000000  1.463850  150.249531   31.646231   
13  LE_HYP_8x12   2           max  3.000000  1.463850  244.365739   57.264519   
14  LE_HYP_8x12   3           avg  3.000000  1.463850  164.387692   45.266070   
15  LE_HYP_8x12   3           max  3.000000  1.463850  271.091101   60.448363   
16  LE_HYP_8x12   4           avg  2.750000  1.544786  130.748163   14.227889   
17  LE_HYP_8x12   4           max  2.750000  1.544786  235.599383   82.045606   
18  LE_HYP_8x12   5           avg  3.000000  1.463850  149.057129   55.932891   
19  LE_HYP_8x12   5           max  3.000000  1.463850  246.295908   94.502639   
20  LE_HYP_8x12   6           avg  3.250000  1.544786  144.847443   64.606802   
21  LE_HYP_8x12   6           max  3.250000  1.544786  206.513535   75.406740   
22  LE_HYP_8x12   7           avg  3.000000  1.463850  147.809510   68.919126   
23  LE_HYP_8x12   7           max  3.000000  1.463850  234.964970  110.251443   
24  LE_HYP_8x12   8           avg  3.000000  1.463850  138.009628   73.184410   
25  LE_HYP_8x12   8           max  3.000000  1.463850  220.902395  112.042654   
26   LE_MAX_5x4   1           avg  3.000000  1.463850  255.492017   97.621258   
27   LE_MAX_5x4   1           max  3.000000  1.463850  385.281046  134.764551   
28   LE_MAX_5x4   2           avg  3.000000  1.651446  278.222444  135.363319   
29   LE_MAX_5x4   2           max  3.000000  1.651446  409.513416  152.142098   
30   LE_MAX_5x4   3           avg  3.000000  1.463850  167.688346  101.216034   
31   LE_MAX_5x4   3           max  3.000000  1.463850  260.165068  160.965386   
32   LE_MAX_5x4   4           avg  3.500000  1.167748  189.048040   35.408602   
33   LE_MAX_5x4   4           max  3.500000  1.167748  296.489682   69.664349   
34   LE_MAX_5x4   5           avg  4.000000  0.866025  186.973726   42.689391   
35   LE_MAX_5x4   5           max  4.000000  0.866025  251.980445   54.130195   
36  LP_HYP_5x12   1           avg  3.000000  1.463850  153.555422   66.319772   
37  LP_HYP_5x12   1           max  3.000000  1.463850  258.299836   95.559191   
38  LP_HYP_5x12   2           avg  3.000000  1.463850  125.291256   56.789040   
39  LP_HYP_5x12   2           max  3.000000  1.463850  235.854833   95.965683   
40  LP_HYP_5x12   3           avg  2.500000  1.167748  145.382777   89.849720   
41  LP_HYP_5x12   3           max  2.500000  1.167748  246.563683  145.108816   
42  LP_HYP_5x12   4           avg  2.666667  1.322876  196.793741  117.688183   
43  LP_HYP_5x12   4           max  2.666667  1.322876  326.501037  176.611891   
44  LP_HYP_5x12   5           avg  2.500000  1.167748  145.064309  103.248038   
45  LP_HYP_5x12   5           max  2.500000  1.167748  267.040705  205.394145   

    B0_std_rf                 B1_rf  ...       B0_vm              B0_std_vm  \
         mean        std       mean  ..

In [13]:
for analysis in ['max', 'avg']:
    for muscle in ['vl', 'rf', 'vm']:
        df_to_use = avg_tabel[avg_tabel['type_analysis'] == analysis]
        parameter_to_use = f'B0_{muscle}'
        
        fig, axes = plt.subplots(2, 2, figsize=(12, 8))
        axes = axes.flatten()

        # Determine the y-axis limits
        y_min = df_to_use[parameter_to_use]['mean'].min() - 0.5 * df_to_use[parameter_to_use]['std'].max()
        y_max = df_to_use[parameter_to_use]['mean'].max() + 0.5 * df_to_use[parameter_to_use]['std'].max()

        for i, (type, group_data) in enumerate(df_to_use.groupby('training')):
            sns.lineplot(data=group_data, x='set', y=group_data[parameter_to_use]['mean'], label='mean', ax=axes[i])
            
            lower_bound = group_data[parameter_to_use]['mean'] - group_data[parameter_to_use]['std'] / 2
            upper_bound = group_data[parameter_to_use]['mean'] + group_data[parameter_to_use]['std'] / 2
            
            axes[i].fill_between(group_data['set'], lower_bound, upper_bound, color='blue', alpha=0.3, label='std')
            axes[i].text(0.5, -0.2, f"mean_R²: {round(group_data[f'R2_{muscle}']['mean'].mean(),4)}", transform=axes[i].transAxes, fontsize=10, ha='center')
            axes[i].set_ylabel('Mean RMS')
            axes[i].set_title(type)
            axes[i].legend()
            
            # Set y-axis limits
            axes[i].set_ylim(y_min, y_max)

        fig.suptitle(f'{analysis} of {parameter_to_use}')
        plt.tight_layout()
        plt.savefig(f'{analysis}_{parameter_to_use}')
